In [1]:
# Created by GB - 10/22
# Google Colab was used in this code
import numpy as np
import pandas as pd
import PIL
import pathlib
import os
import random
import matplotlib.pyplot as plt
import warnings

In [2]:
from google.colab import drive
drive.mount('/content/drive/')
path1 = r'/content/drive/MyDrive/Colab Notebooks/Images/iron man comics_dir_resized'
path2 = r'/content/drive/MyDrive/Colab Notebooks/Images/spider man comics_dir_resized'

Mounted at /content/drive/


In [3]:
# Converting images to numpy arrays 
def numpy_arrays(path1,path2):
    delimiter = chr(92)
    path_list = []
    arrays_list1 = []
    arrays_list2 = []
    path_list.append(path1)
    path_list.append(path2)
    counter = 0
    for paths in range(len(path_list)):
        listdir = os.listdir(path_list[paths])
        counter += 1
        for items in listdir:
            items.split()
            if counter == 1:
                image = PIL.Image.open(path1+'/'+items) #problem with delimiter !WARNING!
            else:
                image = PIL.Image.open(path2+'/'+items)
            image_inf = image.getdata()
            sub_array = np.array(image_inf)
            if counter == 1:
                arrays_list1.append(sub_array)
            else:
                arrays_list2.append(sub_array)
    
    ones= np.ones((len(arrays_list1),), dtype=int)
    ones = ones.tolist()
    zeros= np.zeros((len(arrays_list2),), dtype=int)
    zeros = zeros.tolist()
    return arrays_list1,arrays_list2,ones,zeros

In [4]:
def conc_lists(array1,array2,ones,zeros):
    arrayX = array1
    arrayY = ones
    for i in range(len(array2)):
        arrayX.append(array2[i])
    for x in range(len(zeros)):
        arrayY.append(zeros[x])
        
    return arrayX, arrayY

In [5]:
def shuffle(X1,X2):
    X3 = list(zip(X1,X2))
    random.shuffle(X3)
    X1,X2 = zip(*X3)
    return X1,X2

In [6]:
def train_test_split(array1,array2,train_size):
        len_array1 = len(array1)
        len_array2 = len(array2)
        array1_train_size = int(train_size*len_array1)
        array2_train_size = int(train_size*len_array2)
        X_train = array1[:array1_train_size]
        Y_train = array2[:array1_train_size]
        X_test = array1[array2_train_size:]
        Y_test = array2[array2_train_size:]
        return X_train,Y_train,X_test,Y_test

In [7]:
# Calculating accuracy
def accuracy_s(y_true,y_pred):
    len_of_t = len(y_true)
    len_of_p = len(y_pred)
    true = 0
    if len_of_t != len_of_p:
        print("The sizes are not matched !")
    else:
        for _ in range(len_of_t):
            if y_true[_]==y_pred[_]:
                true += 1
    
    accuracy = true/len_of_t
    return accuracy

In [8]:
    ## Preparing the data ##

# Convert images into numpy arrays #
# Concatenate the arrays #
# Shuffle #
# Split into train and test sets #

In [9]:
array1,array2,ones,zeros=numpy_arrays(path1,path2)
print('class 1 len: ',len(array1))
print('class 1 data type: ',type(array1))
print('class 1 image size (multiplicated): ',array1[1].shape)
print('class 2 len: ',len(array2))
print('class 2 data type: ',type(array2))
print('class 2 image size (multiplicated): ',array2[1].shape)
print('class 1 y len: ',len(ones))

class 1 len:  656
class 1 data type:  <class 'list'>
class 1 image size (multiplicated):  (160000,)
class 2 len:  656
class 2 data type:  <class 'list'>
class 2 image size (multiplicated):  (160000,)
class 1 y len:  656


In [10]:
# This part is used to concancate class arrays and their y values
arrayX,arrayY = conc_lists(array1,array2,ones,zeros)
len(arrayY)

1312

In [11]:
# This part is used to shuffle all data with same way.
# After running, we have more realistic data to classificate
array_X,array_Y = shuffle(arrayX,arrayY)
len(array_X)

1312

In [12]:
# This part is to split the data into train and test datas.
train_X,train_Y,test_X,test_Y=train_test_split(array_X,array_Y,train_size=0.9)
print('length of X values of train data: ',len(train_X))
print('length of Y values of train data: ',len(train_Y))
print('length of X values of test data: ',len(test_X))
print('length of Y values of test data: ',len(test_Y))
train_X = np.array(train_X)
train_Y = np.array(train_Y)
test_X = np.array(test_X)
test_Y = np.array(test_Y)

length of X values of train data:  1180
length of Y values of train data:  1180
length of X values of test data:  132
length of Y values of test data:  132


In [13]:
# Normalizing the datas #
# Normalizing the datas #
standard_X_train =  (train_X - np.average(train_X)) / (np.std(train_X))
standard_X_test =  (test_X - np.average(test_X)) / (np.std(test_X))

In [ ]:
### Classification with Hard Margin - Support Vector Machine ###

In [14]:
# Parameter initialization #
param_init = {}
param_init["w"] = np.random.randn(train_X.shape[1]) * 0.1
param_init["b"] = 0

In [17]:
def SVM(X,y,parameters,learning_rate,epochs,hmlambda):
    w = parameters["w"]
    b = parameters["b"]
    loss_list = []
    y_l = np.where(y <= 0, -1,1)

    for _ in range(epochs):
        for i, x in enumerate(X):
            cond = y_l[i] * (np.dot(x,w) - b) >= 1
            if cond == True:
                dw = 2 * hmlambda * w
                db = 0
            else:
                dw = 2 * hmlambda * w - np.dot(x,y_l[i])
                db = y_l[i] 

            w = w - learning_rate * dw
            b = b - learning_rate * db
        
    parameters["w"] = w
    parameters["b"] = b
    return parameters

In [18]:
warnings.filterwarnings('ignore')
learning_rate = 0.01
epochs = 500
hmlambda = 0.01
parameters = SVM(standard_X_train,train_Y, param_init, learning_rate, epochs, hmlambda)

In [19]:
out = np.dot(standard_X_test, parameters["w"]) + parameters["b"]
labels = np.sign(out)
y_pred = np.where(labels <= -1,0,1)
accuracy = accuracy_s(test_Y,y_pred)
accuracy

0.45454545454545453

In [20]:
# Sklearn Example 1
from sklearn.preprocessing import StandardScaler
from sklearn import svm
from sklearn.metrics import accuracy_score

svm_class = svm.SVC(kernel='linear')
svm_class.fit(standard_X_train, train_Y)
pred_svm = svm_class.predict(standard_X_test)
accuracy_sk = accuracy_score(test_Y, pred_svm)
accuracy_sk


0.48484848484848486

In [21]:
# Sklearn Example 2
from sklearn.preprocessing import StandardScaler
from sklearn import svm
from sklearn.metrics import accuracy_score

svm_class = svm.SVC(kernel='poly')
svm_class.fit(standard_X_train, train_Y)
pred_svm = svm_class.predict(standard_X_test)
accuracy_sk = accuracy_score(test_Y, pred_svm)
accuracy_sk

0.5227272727272727